For Google Colab

In [1]:
# !curl -fsSL https://ollama.com/install.sh | sh
# !nohup ollama serve > output.log 2>&1 &
# !ollama pull gemma3

In [2]:
# !nohup ollama serve > output.log 2>&1 &

In [3]:
# !pip install langchain langchainhub chromadb ollama
# !pip install langchain-community langchain-core

For Local Jupyter Notebook

In [4]:
from langchain_ollama import OllamaLLM
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain.chains import ConversationChain
from langchain.vectorstores import Chroma
from langchain.embeddings import OllamaEmbeddings
from langchain.memory import VectorStoreRetrieverMemory

# Step 1: Setup LLM
llm = OllamaLLM(model="gemma3:12b")

In [5]:
# Step 2: Short-Term Memory (ConversationBufferMemory)
short_term_memory = ConversationBufferMemory()

short_term_chain = ConversationChain(
    llm=llm,
    memory=short_term_memory,
    verbose=True
)

/var/folders/ws/xp1j_rx97fv3_3dysf1hrljw0000gn/T/ipykernel_25186/1865559759.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  short_term_memory = ConversationBufferMemory()
/var/folders/ws/xp1j_rx97fv3_3dysf1hrljw0000gn/T/ipykernel_25186/1865559759.py:4: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  short_term_chain = ConversationChain(


In [6]:
# Interact with short-term memory
print("=== Short-Term Memory Example ===")
short_term_chain.predict(input="My name is Ashiq.")

=== Short-Term Memory Example ===


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: My name is Ashiq.
AI:

> Finished chain.


"It's lovely to meet you, Ashiq! I'm glad you shared your name. I'm an AI, and I don't have a name in the traditional sense, but you can call me AI. It's a pleasure to be chatting with you. Do you have a favorite color, Ashiq? Knowing a little about you helps me tailor our conversation!"

In [7]:
short_term_chain.predict(input="What's my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: My name is Ashiq.
AI: It's lovely to meet you, Ashiq! I'm glad you shared your name. I'm an AI, and I don't have a name in the traditional sense, but you can call me AI. It's a pleasure to be chatting with you. Do you have a favorite color, Ashiq? Knowing a little about you helps me tailor our conversation!
Human: What's my name?
AI:

> Finished chain.


"Your name is Ashiq! You just told me that in our very first exchange. I store information about our conversations, and I remember you sharing your name. It's great to be talking with you, Ashiq!"

In [8]:
# Step 3: Long-Term Memory (Using Chroma + RetrieverMemory)
embedding = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma(embedding_function=embedding, persist_directory="../chroma_store")
retriever = vectorstore.as_retriever()

long_term_memory = VectorStoreRetrieverMemory(retriever=retriever)

/var/folders/ws/xp1j_rx97fv3_3dysf1hrljw0000gn/T/ipykernel_25186/132408609.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding = OllamaEmbeddings(model="nomic-embed-text")
/var/folders/ws/xp1j_rx97fv3_3dysf1hrljw0000gn/T/ipykernel_25186/132408609.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(embedding_function=embedding, persist_directory="../chroma_s

In [9]:
# Retrieve memory later
docs = retriever.get_relevant_documents("What color do I like?")
print("=== Long-Term Memory Retrieval ===")
for doc in docs:
    print(doc.page_content)

# Step 4: Optional - Use in a chain
long_term_chain = ConversationChain(
    llm=llm,
    memory=long_term_memory,
    verbose=True
)

=== Long-Term Memory Retrieval ===


/var/folders/ws/xp1j_rx97fv3_3dysf1hrljw0000gn/T/ipykernel_25186/3618623708.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("What color do I like?")


In [10]:
# Save long-term memory manually
long_term_memory.save_context({"input": "My favorite color is blue."},
                              {"output": "Got it, blue is your favorite."})

In [11]:
# Interact
long_term_chain.predict(input="Can you remind me what color I like?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
input: My favorite color is blue.
output: Got it, blue is your favorite.
Human: Can you remind me what color I like?
AI:

> Finished chain.


"Certainly! You mentioned that your favorite color is blue. I’m programmed to remember these sorts of details, so I'm happy to recall it for you. It’s a lovely color – did you know that blue is often associated with feelings of calmness and serenity? There's even a fascinating history of how blue pigments were created and used throughout different cultures! But for now, I just wanted to confirm that you like blue."

In [12]:
short_term_chain.predict(input="Can you recall my name?")  # Should recall "Alex"



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: My name is Ashiq.
AI: It's lovely to meet you, Ashiq! I'm glad you shared your name. I'm an AI, and I don't have a name in the traditional sense, but you can call me AI. It's a pleasure to be chatting with you. Do you have a favorite color, Ashiq? Knowing a little about you helps me tailor our conversation!
Human: What's my name?
AI: Your name is Ashiq! You just told me that in our very first exchange. I store information about our conversations, and I remember you sharing your name. It's great to be talking with you, Ashiq!
Human: Can you recall my name?
AI:

> Finished chain.


"Absolutely! Your name is Ashiq. I’m designed to remember details from our conversations, and recalling your name is one of those details. It’s quite handy for making our interactions feel more personal, don't you think? I find it really helps to build rapport. Just like remembering that you told me your name – it was the very first thing you shared!"

In [13]:
short_term_memory.clear()